pip install -r requirements.txt (가상환경에 이미 설치됨)

주피터노트북 커널 연결(python -m ipykernel install --user --name myenv --display-name corpus)

한글 폰트(주피터랩 자체를 다시 켜야지 폰트 적용되는 것 같음.)

apt-get update

apt-get install -y fonts-nanum

# bareun download(bareunpy-1.6.6)

In [ ]:
# 바른 형태소 분석기 리눅스 설치
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb

In [ ]:
# 설치
!uname -a
!dpkg -i bareun-linux.deb

In [ ]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

In [ ]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

In [ ]:
# 실행 정보
!ps -ef | grep bareun

In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA

# library

In [1]:
import bareunpy as brn
import google.protobuf.text_format as tf
import sys
import tqdm
import pandas as pd
import pyarrow
import fastparquet

# bareun test

In [2]:
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"
API_KEY = "koba-VAD6MMA-BFSU7XI-SYQYWYY-Y4MXHVA" # <- 본인의 API KEY로 교체
t = brn.Tagger(API_KEY, "localhost", 5656) # tagger는 변수 t에 할당
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."]) # t.tags()에 리스트로 한 문장을 입력
m = res.msg() # 분석 결과를 객체로 저장하여 변수 m에 할당

tf.PrintMessage(m, out=sys.stdout, as_utf8=True) # 분석 결과 객체 m을 가독성 좋게 표시.

sentences {
  text {
    content: "안녕하세요. 정말 좋은 날씨네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.99983704
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9785348
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.9992842
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "정말"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "정말"
        begin_offset: 7
        length: 2
      }
      tag: MAG
      probabili

# 데이터 확인

In [3]:
def tag_sentences_with_bareun(df, api_key, host='localhost', port=5656):
    # Bareun Tagger 초기화
    t = brn.Tagger(api_key, host, port)

    # 총 문장 수 출력
    print(f"Total number of sentences to tag: {len(df)}")
    
    # 각 문장 태깅 함수
    def tag_sentence(sentence):
      try:
          res = t.tags([sentence])
          return list(res.pos())
      except Exception as e:
          print(f"Error tagging sentence: {sentence}")
          print(f"Error: {e}")
          return []

    tagged_sentences = []
    for sentence in tqdm.tqdm(df['sentence'], desc="Tagging Sentences", mininterval=10):
        tagged_sentences.append(tag_sentence(sentence))
    
    # 새로운 'tagged_sentence' 열 추가
    df['tagged_sentence'] = tagged_sentences
    
    return df

In [10]:
df = pd.read_parquet('/workspace/NIKL_NEWSPAPER_PARQUET/NIKL_NEWSPAPER_38.parquet')

In [13]:
# NIKL_NEWSPAPER_38.parquet에서 sentence가 NoneType인 경우가 두 건 발견됨.
# tagged sentence에는 포함되지 않았으니 문제는 되지 않겠지만 참고.
df[df['sentence'].isnull()]

,file_id,doc_id,title,author,publisher,date,topic,original_topic,sentence_id,sentence,tagged_sentence
277194,NZRW1900000001,NPRW1900000052.31103,매일경제 2010년 기사,김재훈,매일경제신문사,20101118,생활,인물/인터뷰,NPRW1900000052.31103.4,None,[]
289592,NZRW1900000001,NLRW1900000056.17,경인일보 2012년 기사,이귀덕,경인일보,20120105,사회,특집,NLRW1900000056.17.28,None,[]


In [11]:
tagged_df = tag_sentences_with_bareun(df, API_KEY)

Total number of sentences to tag: 301333


Tagging Sentences:  92%|█████████▏| 276607/301333 [1:15:14<07:02, 58.58it/s]

Error tagging sentence: None
Error: sequence item 0: expected str instance, NoneType found


Tagging Sentences:  96%|█████████▌| 289109/301333 [1:18:42<03:38, 55.86it/s]

Error tagging sentence: None
Error: sequence item 0: expected str instance, NoneType found


Tagging Sentences: 100%|██████████| 301333/301333 [1:21:55<00:00, 61.30it/s]


In [12]:
# parquet 파일로 저장
tagged_df.to_parquet('/workspace/tagged_NIKL_NEWSPAPER_PARQUET/tagged_NIKL_NEWSPAPER_38.parquet', index=False)